In [16]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

In [25]:
url = "https://jagular.iisg.agh.edu.pl/~jdajda/photos/"

# create directory
# os.mkdir("photos")

resp = requests.get(url).content
soup = BeautifulSoup(resp, 'html.parser')
links = soup.find_all("a")

links = list(filter(lambda x: x["href"].endswith(".jpg"), links))

for link in links:
    with open(f"photos/{link['href']}", "wb") as file:
        response = requests.get(f"{url}/{link['href']}")
        file.write(response.content)

In [27]:
from exif import Image
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles

files = os.listdir("photos")
print(files)


['GoogleGlass1.jpg', 'GooglePixel3.jpg', 'GooglePixel4XL.jpg', 'GooglePixel4.jpg', 'GooglePixel3XL.jpg', 'GooglePixelXL.jpg', 'GooglePixel3aXL.jpg', 'GooglePixel2XL.jpg', 'GoogleNexusS.jpg', 'GooglePixel.jpg', 'GooglePixel2.jpg']


In [31]:
from ipyleaflet import Map, Polyline
import os
import collections

def get_sorted_coordinates(root):
    coordinates = {}
    for file in os.listdir(root):
        path = os.path.join(root, file)
        creation_date = os.stat(path).st_mtime
        with open(path, 'rb') as image_file:
            my_image = Image(image_file)
            try:
                coordinates[file] = my_image.gps_latitude[0], my_image.gps_longitude[0], creation_date
            except Exception:
                coordinates[file] = None, None, 0
    sorted_coordinates = sorted(coordinates.items(), key=lambda x: x[1][2])
    return collections.OrderedDict(sorted_coordinates)

data = get_sorted_coordinates("photos")

m = Map(
    basemap=basemaps.CartoDB.Positron,
    center=(52.204793, 0.0),
    zoom=4
)

for file_name,coodinates in data.items():
    if coodinates[0]:
        m.add_layer(Marker(location=coodinates, title=file_name))

lst_coordinates = [[v[0],v[1]] for k,v in data.items() if v[0]]

lines = Polyline(
    locations=[lst_coordinates],
    color="green" ,
    fill=False
)
m.add_layer(lines)
m.save('photos/path_from_photos.html', title='My Map')
